In [ ]:
import cv2
import numpy as np
from pathlib import Path
import yaml

from hfnet.datasets.robotcar import Robotcar
from hfnet.evaluation.localization import Localization
from hfnet.evaluation.utils.db_management import read_query_list
from hfnet.evaluation.loaders import export_loader
from hfnet.evaluation.visualize import draw_matches, draw_keypoints
from hfnet.settings import DATA_PATH, EXPER_PATH
from utils import plot_images

%load_ext autoreload
%autoreload 2
%matplotlib inline

# Run one of the three configs

### NV+SP

In [ ]:
config_global = {
    'db_name': 'globaldb_netvlad.pkl',
    'experiment': 'netvlad/robotcar_resize-960',
    'predictor': export_loader, 
    'has_keypoints': False, 
    'has_descriptors': False, 
    'pca_dim': 1024,
    'num_prior': 10,
}
config_local = {
    'db_name': 'localdb_sp-nms4_focal-from-ref-db_shift-kpts.pkl',
    'experiment': 'super_point_pytorch/robotcar_resize-960',
    'predictor': export_loader,
    'has_keypoints': True,
    'has_descriptors': True,
    'binarize': False,
    'do_nms': True,
    'nms_thresh': 4,
    'num_features': 2000,
    'ratio_thresh': 0.9,
}
model = 'sp_model'

### HF-Net

In [ ]:
config_global = {
    'db_name': 'older/globaldb_hf_glm-bdd_lrsteps_focal-from-ref-db_shift-kpts.pkl',
    'experiment': 'hfnet-shared_weights-unc_aug-photo_glm-bdd_lrsteps/robotcar_resize-960',
    'predictor': export_loader, 
    'has_keypoints': False, 
    'has_descriptors': False, 
    'pca_dim': 1024,
    'num_prior': 10,
}
config_local = {
    'db_name': 'localdb_hf_glm-bdd_lrsteps.pkl',
    'experiment': 'hfnet-shared_weights-unc_aug-photo_glm-bdd_lrsteps/robotcar_resize-960',
    'predictor': export_loader,
    'has_keypoints': True,
    'has_descriptors': True,
    'binarize': False,
    'do_nms': True,
    'nms_thresh': 4,
    'num_features': 2000,
    'ratio_thresh': 0.9,
}
model = 'sp_model'

### NV+SIFT

In [ ]:
config_global = {
    'db_name': 'globaldb_netvlad.pkl',
    'experiment': 'netvlad/robotcar_resize-960',
    'predictor': export_loader, 
    'has_keypoints': False, 
    'has_descriptors': False, 
    'pca_dim': 1024,
    'num_prior': 10,
}
config_local = {
    'db_name': 'localdb_sift_raw.pkl',
    'colmap_db': 'overcast-reference.db',
    'colmap_db_queries': 'query.db',
    'broken_db': True,
    'ratio_thresh': 0.7,
}
model = 'sift_model'

# Setup localization

In [ ]:
config_pose = {
    'reproj_error': 12,
    'min_inliers': 15,
}
config = {'global': config_global, 'local': config_local, 'pose': config_pose}
loc = Localization('robotcar', model, config)
queries = read_query_list(Path(loc.base_path, 'queries/night_queries_with_intrinsics.txt'))

# Optionally: analyze failure modes

In [ ]:
eval_file = 'robotcar/eval_name.yaml'
with open(Path(EXPER_PATH, 'eval', eval_file), 'r') as f:
    failures = yaml.load(f)['metrics']['failure']
queries = [queries[f] for f in failures]  # failures
#queries = [queries[i] for i in range(len(queries)) if i not in set(failures)]  # success

# Ready query dataset

In [ ]:
query_dataset = Robotcar(**{'resize_max': 960, 'image_names': [q.name for q in queries]})
def get_image(name):
    path = Path(DATA_PATH, query_dataset.dataset_folder, name)
    return cv2.imread(path.as_posix())

# Localize

In [ ]:
query_iter = query_dataset.get_test_set()
for i, query_info, query_data in zip(range(10), queries, query_iter):
    results, debug = loc.localize(query_info, query_data, debug=True)
    s = f'{i} {"Success" if results.success else "Failure"}, inliers {results.num_inliers:^4}, ' \
        + f'ratio {results.inlier_ratio:.3f}, landmarks {len(debug["matching"]["lm_frames"]):>4}, ' \
        + f'spl {debug["index_success"]:>2}, places {[len(p) for p in debug["places"]]:}, ' \
        + f'pos {[f"{n:.1f}" for n in results.T[:3, 3]]}'
    print(s)
    
    sorted_frames, counts = np.unique(
        [debug['matching']['lm_frames'][m2] for m1, m2 in debug['matches'][debug['inliers']]],
        return_counts=True)
    best_id = sorted_frames[np.argmax(counts)]

    query_image = get_image(query_info.name)
    best_image = get_image(loc.images[best_id].name)
    best_matches_inliers = [(m1, debug['matching']['lm_indices'][m2]) 
                            for m1, m2 in debug['matches'][debug['inliers']] 
                            if debug['matching']['lm_frames'][m2] == best_id]
    best_matches_outliers = [(m1, debug['matching']['lm_indices'][m2])
                            for i, (m1, m2) in enumerate(debug['matches']) 
                            if debug['matching']['lm_frames'][m2] == best_id
                            and i not in debug['inliers']]
    pair_inliers = draw_matches(query_image, debug['query_item'].keypoints,
                                best_image, loc.local_db[best_id].keypoints,
                                best_matches_inliers, color=(0, 255, 0))
    pair = draw_matches(query_image, debug['query_item'].keypoints,
                        best_image, loc.local_db[best_id].keypoints,
                        best_matches_outliers+best_matches_inliers, 
                        color=(([(0, 0, 255)]*len(best_matches_outliers))
                               +([(0, 255, 0)]*len(best_matches_inliers))))
    plot_images([pair_inliers], dpi=150)
    plot_images([pair], dpi=150)
    
    plot_images([get_image(loc.images[j].name) 
                 for j in debug['places'][max(debug["index_success"], 0)][:5]],
                ylabel='retrieved')